### Getting the tools Ready:

In [82]:
import urduhack
#urduhack.download()

import pandas as pd
from urduhack.preprocessing import *
from urduhack.normalization import normalize
import json

#for tokenizing
import nltk
import string
import re
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_colwidth",200)

### Custom Functions For Data Cleaning

In [83]:
def text_preprocessing(text):
    """
    This Functions takes a string as input and returns preprocessed form of it in urdu language.
    """
    # removing digits and english letters
    Text=re.sub(r"[a-zA-z0-9]+", "", text.lower())
    
    #removing \n (new line tag)
    Text=Text.replace("\n","") 
    
    #using urduhack's functions for data preprocessing 
    Text=normalize_whitespace(Text)
    Text=remove_punctuation(Text)
    Text=remove_accents(Text)
    Text=replace_urls(Text)
    Text=replace_emails(Text)
    Text=replace_phone_numbers(Text)
    Text=replace_numbers(Text)
    return Text

In [84]:
def normalization(text):
    """
    This Functions takes a string as input and returns normalized, consistent and cleaned form of it in urdu language.
    """
    
    #using urduhack's functions to normalize text. 
    #refer to docs here: https://docs.urduhack.com/en/stable/reference/normalization.html
    Text=normalize(text)
    
    #removing irregular whitescapes/empty srings/larger strings and imputing them with NaN
    Text=Text.strip()
    if len(Text)==0 :
        Text=Text.replace("", "NaN")
        
        return float(Text)
    else:
        return Text

In [85]:
sw=pd.read_json("stopwords-ur.json.txt")[0].tolist()
def remove_stopwords(tokenized_text):
    Text=[w for w in tokenized_text if w not in sw]
    return Text

### Data Preprocessing

In [87]:
#reading from csv (scraped from web)
df=pd.read_csv("army_unlab_v2.csv")

In [88]:
df.head()

,user_name,tweets
0,AAlirazabutt,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں
1,پاکستان ذندہ باد۔ پاک فوج ذندہ باد,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔
2,Opinionator,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔
3,Syed Nazar Abbas,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...
4,Dibarjan,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد


In [89]:
#inspecting the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4497 entries, 0 to 4496
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_name  4497 non-null   object
 1   tweets     4497 non-null   object
dtypes: object(2)
memory usage: 70.4+ KB


In [90]:
#droping non-useful columns
df.drop(["user_name"],axis=1,inplace=True)

In [91]:
df.head()

,tweets
0,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں
1,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔
2,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔
3,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...
4,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد


##### Cleaning and Normalizing

In [92]:
df["cleaned"]=df["tweets"].apply(lambda x: text_preprocessing(x))

In [93]:
df.head()

,tweets,cleaned
0,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں
1,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں
2,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں
3,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہا...
4,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد


In [94]:
df["normalized"]=df["cleaned"].apply(lambda x : normalization(x))

In [95]:
df[:200]

,tweets,cleaned,normalized
0,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں
1,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں
2,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں
3,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہاجر ک...
4,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد
5,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے- اصل بات کارکردگی ہے، \n پاک فوج کے خلاف منفی بات بلکل \nنہیں ہونی چاہئے، صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو، ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...
6,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا، لعنت ہے,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا لعنت ہے,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا لعنت ہے
7,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے \n@asmashirazi\n @hinaparvezbutt\n @Karachi__Wali,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے
8,میں لندن سے نواز شریف کی طرف سےبھج رہا ہوں اور میاں صاحب نے یہ پیغام میر ایاز صادق کو دینا۔کہنہ ارے کتے تو اپنی پارٹی کر ہوکے مجھے ہی دیوار سے لگانے اور میرے سے بڑا غدار اور نمک حرام بنے کی کوشش ک...,میں لندن سے نواز شریف کی طرف سےبھج رہا ہوں اور میاں صاحب نے یہ پیغام میر ایاز صادق کو دیناکہنہ ارے کتے تو اپنی پارٹی کر ہوکے مجھے ہی دیوار سے لگانے اور میرے سے بڑا غدار اور نمک حرام بنے کی کوشش کر...,میں لندن سے نواز شریف کی طرف سےبھج رہا ہوں اور میاں صاحب نے یہ پیغام میر ایاز صادق کو دیناکہنہ ارے کتے تو اپنی پارٹی کر ہوکے مجھے ہی دیوار سے لگانے اور میرے سے بڑا غدار اور نمک حرام بنے کی کوشش کر...
9,فیک نہی ھے اکاونٹ اس کا سافٹ ویر چینج کر دیا ھے فوج نے,فیک نہی ھے اکاونٹ اس کا سافٹ ویر چینج کر دیا ھے فوج نے,فیک نہی ھے اکاونٹ اس کا سافٹ ویر چینج کر دیا ھے فوج نے


In [96]:
#inspecting for missing Values 
df.isna().sum()

tweets          0
cleaned         0
normalized    252
dtype: int64

In [97]:
#droping missing rows
df.dropna(inplace=True,axis=0)

In [98]:
#inspecting again
df.isna().sum()

tweets        0
cleaned       0
normalized    0
dtype: int64

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4245 entries, 0 to 4496
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweets      4245 non-null   object
 1   cleaned     4245 non-null   object
 2   normalized  4245 non-null   object
dtypes: object(3)
memory usage: 132.7+ KB


In [100]:
#tokenizing cleaned text for vectorizing
df["tokenized"]=df.normalized.apply(lambda x : nltk.word_tokenize(x))

In [101]:
df[:300]

,tweets,cleaned,normalized,tokenized
0,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,"[فوج, سے, بڑا, کمینہ, محکمہ, پاکستان, میں, نہیں, ایک, نمبر, کے, بھگوڑے, ہیں]"
1,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں,"[کیوںنکہ, سردی, کی, وجہ, سے, مرغی, کو, سردیوں, میں, باہر, جانے, سےڈر, لگتاہیں]"
2,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں,"[کرتوت, کسی, اور, کے, اور, نام, لگ, جاتا, ہے, فوج, کا, اپوزیشن, کو, یہاں, سے, سمجھنا, چاہیے, کہ, یہ, لوگ, فوج, کے, نام, کے, پیچھے, چھپ, کر, اپنی, کاروای, کرتے, ہیں]"
3,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہاجر ک...,"[سوال, یہ, ہے, کہ, فوج, کی, بہادری, صرف, کراچی, کے, جوانوں, پہ, ہی, صادق, ہوتی, ہے, ایاز, صادق, ہو, یا, کوئی, اور, لیگی, ان, کی, چار, دیواری, تک, جاتے, بوٹ, ٹوٹ, جاتے, ہیں, کیا, بند, کرو, سیاسی, ڈ..."
4,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,"[حوالدار, صٻب, زان, تٻار, ساتہ, پتلون, دی, تلے, واخلہ۰, ناپاک, فوج, مردہ, باد]"
5,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے- اصل بات کارکردگی ہے، \n پاک فوج کے خلاف منفی بات بلکل \nنہیں ہونی چاہئے، صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو، ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...,"[یہ, حالات, جان, بوجھ, کر, پیدا, کئے, گئے, ہیں, تاکہ, لوگوں, کو, حقائق, سےدور, رکھا, جائے, اصل, بات, کارکردگی, ہے, پاک, فوج, کے, خلاف, منفی, بات, بلکل, نہیں, ہونی, چاہئے, صحیح, تنقید, ضرور, کریں, ..."
6,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا، لعنت ہے,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا لعنت ہے,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا لعنت ہے,"[افسوس, کی, بات, ہے, کہ, ایاز, میر, صادق, کے, فوج, کے, خلاف, بیان, پر, نہیں, بلکہ, ثنااللہ, زھری, کو, اسٹیج, پر, نہ, بٹھانے, پر, اختلاف, ہوا, تھا, لعنت, ہے]"
7,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے \n@asmashirazi\n @hinaparvezbutt\n @Karachi__Wali,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے,پھر بولتے ہیں فوج ٹھیک نا ہی ہے اب کیا کرے فوج انکو مارتی ہے یہ کہتے ہیں ہم پے ظلم ہو رہا ہے,"[پھر, بولتے, ہیں, فوج, ٹھیک, نا, ہی, ہے, اب, کیا, کرے, فوج, انکو, مارتی, 

### Removing stopwords

In [102]:
df["sw_removed"]=df.tokenized.apply(lambda x: remove_stopwords(x))

In [103]:
df[:100]

,tweets,cleaned,normalized,tokenized,sw_removed
0,@OfficialDGISPR\n \nفوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,فوج سے بڑا کمینہ محکمہ پاکستان میں نہیں ایک نمبر کے بھگوڑے ہیں,"[فوج, سے, بڑا, کمینہ, محکمہ, پاکستان, میں, نہیں, ایک, نمبر, کے, بھگوڑے, ہیں]","[فوج, سے, بڑا, کمینہ, محکمہ, پاکستان, میں, نہیں, نمبر, بھگوڑے]"
1,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں۔,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں,کیوںنکہ سردی کی وجہ سے مرغی کو سردیوں میں باہر جانے سےڈر لگتاہیں,"[کیوںنکہ, سردی, کی, وجہ, سے, مرغی, کو, سردیوں, میں, باہر, جانے, سےڈر, لگتاہیں]","[کیوںنکہ, سردی, وجہ, سے, مرغی, کو, سردیوں, میں, باہر, جانے, سےڈر, لگتاہیں]"
2,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا۔ اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں۔,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں,کرتوت کسی اور کے اور نام لگ جاتا ہے فوج کا اپوزیشن کو یہاں سے سمجھنا چاہیے کہ یہ لوگ فوج کے نام کے پیچھے چھپ کر اپنی کاروای کرتے ہیں,"[کرتوت, کسی, اور, کے, اور, نام, لگ, جاتا, ہے, فوج, کا, اپوزیشن, کو, یہاں, سے, سمجھنا, چاہیے, کہ, یہ, لوگ, فوج, کے, نام, کے, پیچھے, چھپ, کر, اپنی, کاروای, کرتے, ہیں]","[کرتوت, کسی, نام, لگ, جاتا, فوج, کا, اپوزیشن, کو, یہاں, سے, سمجھنا, چاہیے, فوج, نام, پیچھے, چھپ, کر, اپنی, کاروای]"
3,Quote Tweet\n#MatrukaSindh 40% of Province Sindh\n@MatrukaSindh\n · Oct 31\nسوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے۔ ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہا...,سوال یہ ہے کہ فوج کی بہادری صرف کراچی کے جوانوں پہ ہی صادق ہوتی ہے ایاز صادق ہو یا کوئی اور لیگی ان کی چار دیواری تک جاتے بوٹ ٹوٹ جاتے ہیں کیا بند کرو سیاسی ڈرامہ کراچی سمیت کے ہر انچ پہ مہاجر ک...,"[سوال, یہ, ہے, کہ, فوج, کی, بہادری, صرف, کراچی, کے, جوانوں, پہ, ہی, صادق, ہوتی, ہے, ایاز, صادق, ہو, یا, کوئی, اور, لیگی, ان, کی, چار, دیواری, تک, جاتے, بوٹ, ٹوٹ, جاتے, ہیں, کیا, بند, کرو, سیاسی, ڈ...","[سوال, فوج, بہادری, صرف, کراچی, جوانوں, پہ, صادق, ایاز, صادق, یا, لیگی, ان, چار, دیواری, جاتے, بوٹ, ٹوٹ, جاتے, کیا, بند, سیاسی, ڈرامہ, کراچی, سمیت, انچ, پہ, مہاجر, کو, اسکا, اختیار]"
4,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,حوالدار صٻب زان تٻار ساتہ پتلون دی تلے واخلہ۰ ناپاک فوج مردہ باد,"[حوالدار, صٻب, زان, تٻار, ساتہ, پتلون, دی, تلے, واخلہ۰, ناپاک, فوج, مردہ, باد]","[حوالدار, صٻب, زان, تٻار, ساتہ, پتلون, تلے, واخلہ۰, ناپاک, فوج, مردہ, باد]"
5,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے- اصل بات کارکردگی ہے، \n پاک فوج کے خلاف منفی بات بلکل \nنہیں ہونی چاہئے، صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو، ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...,یہ حالات جان بوجھ کر پیدا کئے گئے ہیں تاکہ لوگوں کو حقائق سےدور رکھا جائے اصل بات کارکردگی ہے پاک فوج کے خلاف منفی بات بلکل نہیں ہونی چاہئے صحیح تنقید ضرور کریں کی جو کریکشکن کے لئے ہو کسی کی بھی ...,"[یہ, حالات, جان, بوجھ, کر, پیدا, کئے, گئے, ہیں, تاکہ, لوگوں, کو, حقائق, سےدور, رکھا, جائے, اصل, بات, کارکردگی, ہے, پاک, فوج, کے, خلاف, منفی, بات, بلکل, نہیں, ہونی, چاہئے, صحیح, تنقید, ضرور, کریں, ...","[حالات, جان, بوجھ, کر, پیدا, تاکہ, کو, حقائق, سےدور, رکھا, جائے, اصل, بات, کارکردگی, پاک, فوج, خلاف, منفی, بات, بلکل, نہیں, ہونی, چاہئے, صحیح, تنقید, جو, کریکشکن, کسی, بھی, بے, عزتی, کرنا, غلط, با..."
6,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹیج پر نہ بٹھانے پر اختلاف ہوا تھا، لعنت ہے,افسوس کی بات ہے کہ ایاز میر صادق کے فوج کے خلاف بیان پر نہیں بلکہ ثنااللہ زھری کو اسٹ

In [104]:
df.to_csv("army_unlab_v2_clean.csv",columns=["normalized"],index=False)

In [105]:
df_ex=pd.read_csv("army_unlab_v2_clean.csv")

In [106]:
df_ex.columns=["Text"]

In [107]:
df_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4245 entries, 0 to 4244
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4245 non-null   object
dtypes: object(1)
memory usage: 33.3+ KB


In [108]:
df_ex.to_excel("army_unlab_v2.xlsx",index=False)